In [2]:
from bs4 import BeautifulSoup
import requests

In [23]:
landing_page = "https://laws-lois.justice.gc.ca/eng/acts/"
opening_page =  requests.get(landing_page)

soup = BeautifulSoup(opening_page.text, features="html.parser")

In [24]:
# printing the HTML layout of the page
print(soup.prettify())

ï»¿
<!DOCTYPE html>
<!--[if lt IE 9]><html class="no-js lt-ie9" lang="en" dir="ltr"><![endif]-->
<!--[if gt IE 8]><!-->
<html class="no-js" dir="ltr" lang="en">
 <!--<![endif]-->
 <head>
  <meta charset="utf-8"/>
  <!-- Web Experience Toolkit (WET) / BoÃ®te Ã  outils de l'expÃ©rience Web (BOEW)
wet-boew.github.io/wet-boew/License-en.html / wet-boew.github.io/wet-boew/Licence-fr.html -->
  <title>
   Consolidated Acts
  </title>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <!-- Meta data -->
  <meta content="Federal laws of canada" name="description"/>
  <meta content="Consolidated federal laws of canada, Consolidated Acts" name="dcterms.title"/>
  <meta content="Legislative Services Branch" name="dcterms.creator" title="Department of Justice"/>
  <meta content="2015-08-18" name="dcterms.issued" title="W3CDTF"/>
  <meta content="2015-08-18" name="dcterms.modified" title="W3CDTF"/>
  <meta content="Consolidated federal laws of canada" name="dcterms.subject" t

In [32]:
# finding all links for alphabetical list of acts

alpha_links = []
for item in soup.find_all("a"):
  try:
    if item["class"] == ["btn", "btn-default"]:
      alpha_page = item["href"]
      print(item)
      alpha_links.append(f"{landing_page}/{alpha_page}")
  except: 
    continue
  

<a class="btn btn-default" href="A.html"><span class="wb-invisible">Acts beginning with</span>A</a>
<a class="btn btn-default" href="B.html"><span class="wb-invisible">Acts beginning with</span>B</a>
<a class="btn btn-default" href="C.html"><span class="wb-invisible">Acts beginning with</span>C</a>
<a class="btn btn-default" href="D.html"><span class="wb-invisible">Acts beginning with</span>D</a>
<a class="btn btn-default" href="E.html"><span class="wb-invisible">Acts beginning with</span>E</a>
<a class="btn btn-default" href="F.html"><span class="wb-invisible">Acts beginning with</span>F</a>
<a class="btn btn-default" href="G.html"><span class="wb-invisible">Acts beginning with</span>G</a>
<a class="btn btn-default" href="H.html"><span class="wb-invisible">Acts beginning with</span>H</a>
<a class="btn btn-default" href="I.html"><span class="wb-invisible">Acts beginning with</span>I</a>
<a class="btn btn-default" href="J.html"><span class="wb-invisible">Acts beginning with</span>J</a>


In [ ]:
print(len(alpha_links))

https://laws-lois.justice.gc.ca/eng/acts//A.html
